## In-Vantage Scoring via PMML

As part of the PIMA Diabetes model, we produce a PMML representation of the xgboost model with the following code

```python
from nyoka import xgboost_to_pmml
...
xgboost_to_pmml(pipeline=model, col_names=feature_names, target_name=target_name, pmml_f_name="artifacts/output/model.pmml")
```

You can deploy that In-Vantage via the AnalyticOps UI, just as with RESTFul and Batch deployments. This notebook shows you how you can then use this to score models via SQL. 

In [1]:
import os
import pandas as pd
import getpass

from teradataml import create_context, remove_context
from teradataml.context.context import get_connection

In [3]:
password = getpass.getpass("Password")

engine = create_context(host="3.238.151.85", username="AOA_DEMO", password=password)
conn = get_connection()
cursor = engine.raw_connection().cursor()

AOA_DEMO Password ··········


## View Published Models

We can view the models published to vantage by querying the table they are published to. Note this information is available via the AOA APIs also

In [4]:
pd.options.display.max_colwidth = 250
pd.read_sql("select * from aoa_ivsm_models", conn)

,model_version,model_id,project_id,deployed_at,model
0,f7741c1f-ca5e-4c16-a1fb-1bd4ea7e3df4,07701a36-8ea1-4156-94be-20836f38d0a7,23e1df4b-b630-47a1-ab80-7ad5385fcd8d,2020-11-23 21:08:42.460,"b'<?xml version=""1.0"" encoding=""UTF-8"" standalone=""yes""?>\n<PMML xmlns=""http://www.dmg.org/PMML-4_4"" xmlns:data=""http://jpmml.org/jpmml-model/InlineTable"" version=""4.3"">\n\t<Header>\n\t\t<Application name=""JPMML-SkLearn"" version=""1.6.3""/>\n\t\t<T..."
1,84cd9235-2cce-47f5-9c3d-3751d89b4520,03c9a01f-bd46-4e7c-9a60-4282039094e6,23e1df4b-b630-47a1-ab80-7ad5385fcd8d,2020-11-26 11:32:49.530,"b'<?xml version=""1.0"" encoding=""UTF-8""?>\n<PMML xmlns=""http://www.dmg.org/PMML-4_4"" version=""4.4"">\n <Header copyright=""Copyright (c) 2018 Software AG"" description=""Default Description"">\n <Application name=""Nyoka"" version=""4.3.0""/>\n ..."
2,38e4693b-141b-4230-892c-a9affbf69671,03c9a01f-bd46-4e7c-9a60-4282039094e6,23e1df4b-b630-47a1-ab80-7ad5385fcd8d,2020-11-26 12:52:09.040,"b'<?xml version=""1.0"" encoding=""UTF-8""?>\n<PMML xmlns=""http://www.dmg.org/PMML-4_4"" version=""4.4"">\n <Header copyright=""Copyright (c) 2018 Software AG"" description=""Default Description"">\n <Application name=""Nyoka"" version=""4.3.0""/>\n ..."
3,80b3502f-a171-4c33-a524-92e43fd59545,03c9a01f-bd46-4e7c-9a60-4282039094e6,23e1df4b-b630-47a1-ab80-7ad5385fcd8d,2020-11-24 12:21:22.520,"b'<?xml version=""1.0"" encoding=""UTF-8""?>\n<PMML xmlns=""http://www.dmg.org/PMML-4_4"" version=""4.4"">\n <Header copyright=""Copyright (c) 2018 Software AG"" description=""Default Description"">\n <Application name=""Nyoka"" version=""4.3.0""/>\n ..."
4,5ef8f6cc-b934-46f1-81e5-b629e52241a3,03c9a01f-bd46-4e7c-9a60-4282039094e6,23e1df4b-b630-47a1-ab80-7ad5385fcd8d,2020-11-24 11:20:52.200,"b'<?xml version=""1.0"" encoding=""UTF-8"" standalone=""yes""?>\n<PMML xmlns=""http://www.dmg.org/PMML-4_4"" xmlns:data=""http://jpmml.org/jpmml-model/InlineTable"" version=""4.4"">\n\t<Header>\n\t\t<Application name=""JPMML-SkLearn"" version=""1.6.7""/>\n\t\t<T..."
5,7d4bf85a-b58e-4e5d-a9e3-0b73c777ce31,03c9a01f-bd46-4e7c-9a60-4282039094e6,23e1df4b-b630-47a1-ab80-7ad5385fcd8d,2020-11-23 13:00:21.920,"b'<?xml version=""1.0"" encoding=""UTF-8"" standalone=""yes""?>\n<PMML xmlns=""http://www.dmg.org/PMML-4_4"" xmlns:data=""http://jpmml.org/jpmml-model/InlineTable"" version=""4.4"">\n\t<Header>\n\t\t<Application name=""JPMML-SkLearn"" version=""1.6.7""/>\n\t\t<T..."


## On-Demand Scoring

In [6]:
query="""
SELECT * FROM IVSM_SCORE(
    on PIMA_TEST
    on (SELECT model_id, model FROM aoa_ivsm_models WHERE model_version = '84cd9235-2cce-47f5-9c3d-3751d89b4520') AS ModelTable DIMENSION
    using
        ModelID('03c9a01f-bd46-4e7c-9a60-4282039094e6')
        ColumnsToPreserve('PatientId')
        ModelType('PMML')
) sc;
"""
pd.read_sql(query, conn).head()

,PatientId,score_result
0,1,"{""probability_0"":0.9886043799233987,""probability_1"":0.011395620076601292,""predicted_HasDiabetes"":0}"
1,3,"{""probability_0"":0.9980198370119622,""probability_1"":0.0019801629880378813,""predicted_HasDiabetes"":0}"
2,8,"{""probability_0"":0.05094995570429306,""probability_1"":0.9490500442957069,""predicted_HasDiabetes"":1}"
3,9,"{""probability_0"":0.13574050732643506,""probability_1"":0.8642594926735649,""predicted_HasDiabetes"":1}"
4,13,"{""probability_0"":0.5286287238760723,""probability_1"":0.4713712761239277,""predicted_HasDiabetes"":0}"
